In [1]:
import numpy as np
import torch
import torchvision.transforms as transforms
from torchvision import datasets
from torch.utils.data.sampler import SubsetRandomSampler

In [5]:
transform = transforms.ToTensor()

train_data = datasets.FashionMNIST(root = 'data', train = True,
                download = True, transform = transform)
test_data = datasets.FashionMNIST(root = 'data', train = True,
                download = True, transform = transform)

train_data[0][0].shape

torch.Size([1, 28, 28])

In [6]:
l = len(train_data)
indices = list(range(l))
np.random.shuffle(indices)
split = int(l * 0.85)
train_idx, valid_idx = indices[:split], indices[split:]

train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

train_loader = torch.utils.data.DataLoader(train_data, batch_size = 64, sampler = train_sampler)
valid_loader = torch.utils.data.DataLoader(train_data, batch_size = 64, sampler = valid_sampler)
test_loader = torch.utils.data.DataLoader(test_data, batch_size = 64)


In [17]:
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module) :
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)
        self.pool = nn.MaxPool2d(2,2)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
        self.conv3 = nn.Conv2d(16, 32, 3, padding=0)
        self.drop1 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(5*5*32, 400, bias = True)
        self.drop2 = nn.Dropout(0.5)
        self.fc2 = nn.Linear(400, 200, bias = True)
        self.fc3 = nn.Linear(200,10, bias = True)
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.drop1(F.relu(self.conv3(x)))
        #print(x.shape)
        x = self.drop2(F.relu(self.fc1(torch.flatten(x, start_dim = 1))))
        x = self.drop2(F.relu(self.fc2(x)))
        x = self.fc3(x)
        return x

model = Net()

print(model)
model(train_data[0][0].reshape(1,1,28,28))

Net(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1))
  (drop1): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=800, out_features=400, bias=True)
  (drop2): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=400, out_features=200, bias=True)
  (fc3): Linear(in_features=200, out_features=10, bias=True)
)


tensor([[-0.0496,  0.0182, -0.0137, -0.0761,  0.0578, -0.0974,  0.0099,  0.0270,
          0.0862,  0.0590]], grad_fn=<AddmmBackward>)

In [18]:
loss_f = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr = 0.05)

In [19]:
epocs = 8
valid_loss_min = np.inf

for epoch in range(epocs):
    train_loss = 0.
    valid_loss = 0.
    
    ## train ##
    model.train()
    for data, target in train_loader :
        
        optimizer.zero_grad()          #making grads zero
        preds = model(data)           #predicting class
        loss = loss_f(preds, target)   #calculating loss
        loss.backward()               #back propagation
        optimizer.step()              #adding gradients to params
        train_loss += loss.item() * data.size(0)
    
    ##validate##
    model.eval()
    for data, target in valid_loader :
        
        preds = model(data)
        valid_loss += loss_f(preds, target).item() * data.size(0)
    
    ##avg loss##
    train_loss /= len(train_loader.dataset)
    valid_loss /= len(valid_loader.dataset)
    
    print(f'Epoch {epoch+1} : TrainLoss = {train_loss}, ValidLoss = {valid_loss}')
    
    if valid_loss < valid_loss_min :
        print(f'Better model! {valid_loss_min} ==> {valid_loss}')
        valid_loss_min = valid_loss
        torch.save(model.state_dict(), 'model_FasMNIST.pt')

Epoch 1 : TrainLoss = 1.433085103726387, ValidLoss = 0.12020845024585725
Better model! inf ==> 0.12020845024585725
Epoch 2 : TrainLoss = 0.6856352077325185, ValidLoss = 0.08870885552565257
Better model! 0.12020845024585725 ==> 0.08870885552565257
Epoch 3 : TrainLoss = 0.5455003498951594, ValidLoss = 0.07778032637039821
Better model! 0.08870885552565257 ==> 0.07778032637039821
Epoch 4 : TrainLoss = 0.48565272952715555, ValidLoss = 0.06793159152666728
Better model! 0.07778032637039821 ==> 0.06793159152666728
Epoch 5 : TrainLoss = 0.4424482436180115, ValidLoss = 0.06381408169269562
Better model! 0.06793159152666728 ==> 0.06381408169269562
Epoch 6 : TrainLoss = 0.41731410558223725, ValidLoss = 0.06109838996728261
Better model! 0.06381408169269562 ==> 0.06109838996728261
Epoch 7 : TrainLoss = 0.3965791146834691, ValidLoss = 0.05658179663022359
Better model! 0.06109838996728261 ==> 0.05658179663022359
Epoch 8 : TrainLoss = 0.3784196736892064, ValidLoss = 0.054916665873924894
Better model! 0.

In [22]:
model.load_state_dict(torch.load('model_FasMNIST.pt'))

test_loss = 0.0
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))

model.eval()

size = len(test_loader.dataset)
correct = 0
for data, target in test_loader :
    pred = model(data)
    correct += ((nn.functional.softmax(pred)).argmax(1) == target).type(torch.float).sum().item()
print(f'accuracy - {correct/size * 100}')

<ipython-input-22-f0651b11f8a4>:13: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  correct += ((nn.functional.softmax(pred)).argmax(1) == target).type(torch.float).sum().item()


accuracy - 87.03666666666666
